In [1]:
# Import dependencies
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [2]:
# Load cleaned_movies into DataFrame
movies_df = pd.read_csv('movies_final.csv')
movies_df.head()

,Unnamed: 0,name,rating,genre,year,released,score,votes,director,writer,star,country,gross,company,runtime
0,0,The Shining,R,Drama,1980,"June 13, 1980 (United States)",8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,46998772.0,Warner Bros.,146.0
1,1,The Blue Lagoon,R,Adventure,1980,"July 2, 1980 (United States)",5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,58853106.0,Columbia Pictures,104.0
2,2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,"June 20, 1980 (United States)",8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,538375067.0,Lucasfilm,124.0
3,3,Airplane!,PG,Comedy,1980,"July 2, 1980 (United States)",7.7,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,United States,83453539.0,Paramount Pictures,88.0
4,4,Caddyshack,R,Comedy,1980,"July 25, 1980 (United States)",7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,39846344.0,Orion Pictures,98.0


Preprocessing

In [3]:
# Drop Unnamed column
movies_df = movies_df.loc[:,~movies_df.columns.str.contains('^Unnamed')]
movies_df

,name,rating,genre,year,released,score,votes,director,writer,star,country,gross,company,runtime
0,The Shining,R,Drama,1980,"June 13, 1980 (United States)",8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,46998772.0,Warner Bros.,146.0
1,The Blue Lagoon,R,Adventure,1980,"July 2, 1980 (United States)",5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,58853106.0,Columbia Pictures,104.0
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,"June 20, 1980 (United States)",8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,538375067.0,Lucasfilm,124.0
3,Airplane!,PG,Comedy,1980,"July 2, 1980 (United States)",7.7,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,United States,83453539.0,Paramount Pictures,88.0
4,Caddyshack,R,Comedy,1980,"July 25, 1980 (United States)",7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,39846344.0,Orion Pictures,98.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7407,The Call of the Wild,PG,Adventure,2020,"February 21, 2020 (United States)",6.8,42000.0,Chris Sanders,Michael Green,Harrison Ford,Canada,111105497.0,20th Century Studios,100.0
7408,The Eight Hundred,Not Rated,Action,2020,"August 28, 2020 (United States)",6.8,3700.0,Hu Guan,Hu Guan,Zhi-zhong Huang,China,461421559.0,Beijing Diqi Yinxiang Entertainment,149.0
7409,The Quarry,R,Crime,2020,"April 17, 2020 (Mexico)",5.4,2400.0,Scott Teems,Scott Teems,Shea Whigham,United States,3661.0,Prowess Pictures,98.0
7410,Tulsa,PG-13,Comedy,2020,"June 3, 2020 (United States)",5.0,294.0,Scott Pryor,Scott Pryor,Scott Pryor,United States,413378.0,Pryor Entertainment,120.0


In [4]:
# Drop relase date columns
#movies_df = movies_df.drop(columns=['released'], axis=1)
#movies_df.head()

In [5]:
# Load inflation indexer data
inflation_df = pd.read_csv('CPIAUCNS.csv')

# Change object to float
inflation_df['CPIAUCNS_NBD20210101'] = pd.to_numeric(inflation_df['CPIAUCNS_NBD20210101'], errors='coerce')

# Check for nulls
inflation_df['CPIAUCNS_NBD20210101'].isnull().sum()

1

In [6]:
# Drop nulls
inflation_df = inflation_df.dropna()
inflation_df

,DATE,CPIAUCNS_NBD20210101
0,1913-01-01,3.64739
1,1914-01-01,3.69660
2,1915-01-01,3.73043
3,1916-01-01,4.01644
4,1917-01-01,4.73300
...,...,...
104,2017-01-01,90.46013
105,2018-01-01,92.66969
106,2019-01-01,94.34906
107,2020-01-01,95.51294


In [7]:
# Create index multiplier
inflation_df['CPI_Multiplier'] = inflation_df['CPIAUCNS_NBD20210101'].iloc[-1] / inflation_df['CPIAUCNS_NBD20210101']
inflation_df

,DATE,CPIAUCNS_NBD20210101,CPI_Multiplier
0,1913-01-01,3.64739,27.416865
1,1914-01-01,3.69660,27.051886
2,1915-01-01,3.73043,26.806561
3,1916-01-01,4.01644,24.897671
4,1917-01-01,4.73300,21.128248
...,...,...,...
104,2017-01-01,90.46013,1.105459
105,2018-01-01,92.66969,1.079101
106,2019-01-01,94.34906,1.059894
107,2020-01-01,95.51294,1.046979


In [9]:
# Delete everything after year in DATE
inflation_df['DATE'] = inflation_df['DATE'].str.split('-').str[0]
inflation_df

,DATE,CPIAUCNS_NBD20210101,CPI_Multiplier
0,1913,3.64739,27.416865
1,1914,3.69660,27.051886
2,1915,3.73043,26.806561
3,1916,4.01644,24.897671
4,1917,4.73300,21.128248
...,...,...,...
104,2017,90.46013,1.105459
105,2018,92.66969,1.079101
106,2019,94.34906,1.059894
107,2020,95.51294,1.046979


In [10]:
# Delete everything after date in released column
movies_df['released'] = movies_df['released'].str.split('(').str[0]

# Apply pd.to_datatime to released column
movies_df['released'] = pd.to_datetime(movies_df['released']).dt.year
movies_df['released'] = movies_df['released'].astype(str)
movies_df

,name,rating,genre,year,released,score,votes,director,writer,star,country,gross,company,runtime
0,The Shining,R,Drama,1980,1980,8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,46998772.0,Warner Bros.,146.0
1,The Blue Lagoon,R,Adventure,1980,1980,5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,58853106.0,Columbia Pictures,104.0
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,1980,8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,538375067.0,Lucasfilm,124.0
3,Airplane!,PG,Comedy,1980,1980,7.7,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,United States,83453539.0,Paramount Pictures,88.0
4,Caddyshack,R,Comedy,1980,1980,7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,39846344.0,Orion Pictures,98.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7407,The Call of the Wild,PG,Adventure,2020,2020,6.8,42000.0,Chris Sanders,Michael Green,Harrison Ford,Canada,111105497.0,20th Century Studios,100.0
7408,The Eight Hundred,Not Rated,Action,2020,2020,6.8,3700.0,Hu Guan,Hu Guan,Zhi-zhong Huang,China,461421559.0,Beijing Diqi Yinxiang Entertainment,149.0
7409,The Quarry,R,Crime,2020,2020,5.4,2400.0,Scott Teems,Scott Teems,Shea Whigham,United States,3661.0,Prowess Pictures,98.0
7410,Tulsa,PG-13,Comedy,2020,2020,5.0,294.0,Scott Pryor,Scott Pryor,Scott Pryor,United States,413378.0,Pryor Entertainment,120.0


In [12]:
# Change object so string to merge
inflation_df['DATE'] = inflation_df['DATE'].astype(str)

# Rename DATE column in inflation df to released to match prior to merge
inflation_df = inflation_df.rename(columns={'DATE':'released'})
inflation_df

,released,CPIAUCNS_NBD20210101,CPI_Multiplier
0,1913,3.64739,27.416865
1,1914,3.69660,27.051886
2,1915,3.73043,26.806561
3,1916,4.01644,24.897671
4,1917,4.73300,21.128248
...,...,...,...
104,2017,90.46013,1.105459
105,2018,92.66969,1.079101
106,2019,94.34906,1.059894
107,2020,95.51294,1.046979


In [14]:
# Merge dataframes
movies_df = pd.merge(movies_df, inflation_df, how = 'left', on= 'released')
movies_df

,name,rating,genre,year,released,score,votes,director,writer,star,country,gross,company,runtime,CPIAUCNS_NBD20210101,CPI_Multiplier
0,The Shining,R,Drama,1980,1980,8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,46998772.0,Warner Bros.,146.0,30.41237,3.288136
1,The Blue Lagoon,R,Adventure,1980,1980,5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,58853106.0,Columbia Pictures,104.0,30.41237,3.288136
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,1980,8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,538375067.0,Lucasfilm,124.0,30.41237,3.288136
3,Airplane!,PG,Comedy,1980,1980,7.7,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,United States,83453539.0,Paramount Pictures,88.0,30.41237,3.288136
4,Caddyshack,R,Comedy,1980,1980,7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,39846344.0,Orion Pictures,98.0,30.41237,3.288136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7407,The Call of the Wild,PG,Adventure,2020,2020,6.8,42000.0,Chris Sanders,Michael Green,Harrison Ford,Canada,111105497.0,20th Century Studios,100.0,95.51294,1.046979
7408,The Eight Hundred,Not Rated,Action,2020,2020,6.8,3700.0,Hu Guan,Hu Guan,Zhi-zhong Huang,China,461421559.0,Beijing Diqi Yinxiang Entertainment,149.0,95.51294,1.046979
7409,The Quarry,R,Crime,2020,2020,5.4,2400.0,Scott Teems,Scott Teems,Shea Whigham,United States,3661.0,Prowess Pictures,98.0,95.51294,1.046979
7410,Tulsa,PG-13,Comedy,2020,2020,5.0,294.0,Scott Pryor,Scott Pryor,Scott Pryor,United States,413378.0,Pryor Entertainment,120.0,95.51294,1.046979


In [15]:
# Adjust gross revenue for inflation
movies_df['adjusted_gross'] = movies_df['gross']*movies_df['CPI_Multiplier']
movies_df

,name,rating,genre,year,released,score,votes,director,writer,star,country,gross,company,runtime,CPIAUCNS_NBD20210101,CPI_Multiplier,adjusted_gross
0,The Shining,R,Drama,1980,1980,8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,46998772.0,Warner Bros.,146.0,30.41237,3.288136,1.545383e+08
1,The Blue Lagoon,R,Adventure,1980,1980,5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,58853106.0,Columbia Pictures,104.0,30.41237,3.288136,1.935170e+08
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,1980,8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,538375067.0,Lucasfilm,124.0,30.41237,3.288136,1.770250e+09
3,Airplane!,PG,Comedy,1980,1980,7.7,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,United States,83453539.0,Paramount Pictures,88.0,30.41237,3.288136,2.744066e+08
4,Caddyshack,R,Comedy,1980,1980,7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,39846344.0,Orion Pictures,98.0,30.41237,3.288136,1.310202e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7407,The Call of the Wild,PG,Adventure,2020,2020,6.8,42000.0,Chris Sanders,Michael Green,Harrison Ford,Canada,111105497.0,20th Century Studios,100.0,95.51294,1.046979,1.163251e+08
7408,The Eight Hundred,Not Rated,Action,2020,2020,6.8,3700.0,Hu Guan,Hu Guan,Zhi-zhong Huang,China,461421559.0,Beijing Diqi Yinxiang Entertainment,149.0,95.51294,1.046979,4.830985e+08
7409,The Quarry,R,Crime,2020,2020,5.4,2400.0,Scott Teems,Scott Teems,Shea Whigham,United States,3661.0,Prowess Pictures,98.0,95.51294,1.046979,3.832988e+03
7410,Tulsa,PG-13,Comedy,2020,2020,5.0,294.0,Scott Pryor,Scott Pryor,Scott Pryor,United States,413378.0,Pryor Entertainment,120.0,95.51294,1.046979,4.327979e+05


In [15]:
# Determine if year needs binning
years = movies_df['year'].value_counts()
years

2009    200
2002    200
2014    200
2013    200
2005    200
2018    199
2006    199
2007    199
2008    199
2011    199
2012    199
2016    199
2017    199
2004    199
1994    198
1996    198
2010    197
2001    197
2003    197
1995    196
1997    196
2015    196
1991    195
1993    195
1998    194
2000    193
1999    193
1988    191
1992    189
1990    189
1989    186
1987    186
1986    182
2019    180
1985    177
1984    155
1983    127
1982    118
1981    103
1980     80
2020     13
Name: year, dtype: int64

In [16]:
# Determine if genre needs binning
genres = movies_df['genre'].value_counts()
genres

Comedy       2182
Action       1666
Drama        1439
Crime         536
Biography     429
Adventure     419
Animation     331
Horror        304
Fantasy        42
Mystery        20
Thriller       12
Family         10
Romance         8
Sci-Fi          8
Western         3
Music           1
Sport           1
Musical         1
Name: genre, dtype: int64

In [17]:
# Bin genres with less than 5 movies into 'Other'
replace_genre = list(genres[genres < 5].index)

# Replace in DataFrame
for genre in replace_genre:
    movies_df['genre'] = movies_df['genre'].replace(genre,'Other')

# Check to see if binning was successful
movies_df['genre'].value_counts()

Comedy       2182
Action       1666
Drama        1439
Crime         536
Biography     429
Adventure     419
Animation     331
Horror        304
Fantasy        42
Mystery        20
Thriller       12
Family         10
Romance         8
Sci-Fi          8
Other           6
Name: genre, dtype: int64

In [7]:
# Determine which directors need to be removed (directors with only 1 movie)
directors = movies_df['director'].value_counts()
directors

Woody Allen         38
Clint Eastwood      31
Steven Spielberg    27
Directors           25
Ron Howard          24
                    ..
Daniel Myrick        1
Tony Cinciripini     1
Rodney Gibbons       1
Nicole Garcia        1
Kevin Shulman        1
Name: director, Length: 2816, dtype: int64

In [9]:
# Bin directors with less than 1 movie into 'Other'
replace_director = list(directors[directors == 1].index)

# Replace in DataFrame
for director in replace_director:
    movies_df['director'] = movies_df['director'].replace(director,'Other')

# Check to see if binning was successful
movies_df['director'].value_counts()

Other               1463
Woody Allen           38
Clint Eastwood        31
Steven Spielberg      27
Directors             25
                    ... 
Tom Hanks              2
David Hogan            2
Steve Buscemi          2
Matthew Bright         2
Adam Robitel           2
Name: director, Length: 1354, dtype: int64

In [10]:
# Remove directors with only 1 film
#movies_df = movies_df[movies_df['director'].map(movies_df['director'].value_counts()) != 1]
#movies_df

In [11]:
# Determine which stars need to be removed
stars = movies_df['star'].value_counts()
stars

Nicolas Cage         43
Tom Hanks            41
Robert De Niro       41
Denzel Washington    37
Bruce Willis         34
                     ..
Teri Hatcher          1
Pamela Anderson       1
Mike Judge            1
Kim Bodnia            1
Kristina Klebe        1
Name: star, Length: 2652, dtype: int64

In [12]:
# Bin stars with less than 1 movie into 'Other'
replace_star = list(stars[stars == 1].index)

# Replace in DataFrame
for star in replace_star:
    movies_df['star'] = movies_df['star'].replace(star,'Other')

# Check to see if binning was successful
movies_df['star'].value_counts()

Other                1634
Nicolas Cage           43
Tom Hanks              41
Robert De Niro         41
Denzel Washington      37
                     ... 
Shaquille O'Neal        2
Scott Mechlowicz        2
Kelsey Grammer          2
Joseph Fiennes          2
Martha Higareda         2
Name: star, Length: 1019, dtype: int64

In [13]:
# Remove stars with only 1 film
#movies_df = movies_df[movies_df['star'].map(movies_df['star'].value_counts()) != 1]
#movies_df

In [14]:
# Determine which writers need to be removed
writers = movies_df['star'].value_counts()
writers

Other                1634
Nicolas Cage           43
Tom Hanks              41
Robert De Niro         41
Denzel Washington      37
                     ... 
Shaquille O'Neal        2
Scott Mechlowicz        2
Kelsey Grammer          2
Joseph Fiennes          2
Martha Higareda         2
Name: star, Length: 1019, dtype: int64

In [15]:
# Determine which companies need to be removed
companies = movies_df['company'].value_counts()
companies

Universal Pictures              376
Warner Bros.                    332
Columbia Pictures               332
Paramount Pictures              319
Twentieth Century Fox           240
                               ... 
Too Askew Prod. Inc.              1
McFarlane Films                   1
Blue Tulip Productions            1
Cinema Line Film Corporation      1
Pryor Entertainment               1
Name: company, Length: 2242, dtype: int64

In [16]:
# Bin companies with less than 1 movie into 'Other'
replace_company = list(companies[companies == 1].index)

# Replace in DataFrame
for company in replace_company:
    movies_df['company'] = movies_df['company'].replace(company,'Other')

# Check to see if binning was successful
movies_df['company'].value_counts()

Other                  1577
Universal Pictures      376
Warner Bros.            332
Columbia Pictures       332
Paramount Pictures      319
                       ... 
Tribeca Productions       2
Jim Henson Pictures       2
Dogstar Films             2
Mutual Film Company       2
Neverending Media         2
Name: company, Length: 666, dtype: int64

In [17]:
# Remove companies with only 1 film
#movies_df = movies_df[movies_df['company'].map(movies_df['company'].value_counts()) != 1]

In [18]:
# Determine value_counts() for scores
movies_df['score'].value_counts()

6.6    348
6.4    344
6.2    339
6.5    326
6.7    318
      ... 
2.1      2
9.3      1
2.3      1
2.6      1
9.0      1
Name: score, Length: 72, dtype: int64

In [18]:
# Remove scores with only 1 value
movies_df = movies_df[movies_df['score'].map(movies_df['score'].value_counts()) != 1]
# Check to make sure proper scores were removed
movies_df['score'].value_counts()

6.6    348
6.4    344
6.2    339
6.5    326
6.7    318
      ... 
8.9      3
1.9      3
2.1      2
2.4      2
2.8      2
Name: score, Length: 68, dtype: int64

In [19]:
# View preprocessed dataframe
movies_df

,name,rating,genre,year,released,score,votes,director,writer,star,country,gross,company,runtime,CPIAUCNS_NBD20210101,CPI_Multiplier,adjusted_gross
0,The Shining,R,Drama,1980,1980,8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,46998772.0,Warner Bros.,146.0,30.41237,3.288136,1.545383e+08
1,The Blue Lagoon,R,Adventure,1980,1980,5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,58853106.0,Columbia Pictures,104.0,30.41237,3.288136,1.935170e+08
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,1980,8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,538375067.0,Lucasfilm,124.0,30.41237,3.288136,1.770250e+09
3,Airplane!,PG,Comedy,1980,1980,7.7,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,United States,83453539.0,Paramount Pictures,88.0,30.41237,3.288136,2.744066e+08
4,Caddyshack,R,Comedy,1980,1980,7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,39846344.0,Orion Pictures,98.0,30.41237,3.288136,1.310202e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7407,The Call of the Wild,PG,Adventure,2020,2020,6.8,42000.0,Chris Sanders,Michael Green,Harrison Ford,Canada,111105497.0,20th Century Studios,100.0,95.51294,1.046979,1.163251e+08
7408,The Eight Hundred,Not Rated,Action,2020,2020,6.8,3700.0,Hu Guan,Hu Guan,Zhi-zhong Huang,China,461421559.0,Beijing Diqi Yinxiang Entertainment,149.0,95.51294,1.046979,4.830985e+08
7409,The Quarry,R,Crime,2020,2020,5.4,2400.0,Scott Teems,Scott Teems,Shea Whigham,United States,3661.0,Prowess Pictures,98.0,95.51294,1.046979,3.832988e+03
7410,Tulsa,PG-13,Comedy,2020,2020,5.0,294.0,Scott Pryor,Scott Pryor,Scott Pryor,United States,413378.0,Pryor Entertainment,120.0,95.51294,1.046979,4.327979e+05


In [10]:
# Create DataFrame with 'gross' column to merge into .get_dummies() dataframe
numeric_df = movies_df[['name','gross']]
numeric_df

,name,gross
0,The Shining,46998772.0
1,The Blue Lagoon,58853106.0
2,Star Wars: Episode V - The Empire Strikes Back,538375067.0
3,Airplane!,83453539.0
4,Caddyshack,39846344.0
...,...,...
7407,The Call of the Wild,111105497.0
7408,The Eight Hundred,461421559.0
7409,The Quarry,3661.0
7410,Tulsa,413378.0


Machine Learning Model

In [20]:
# Split the data into features and target
# Create our features
#X = pd.get_dummies(movies_df, columns=['rating','genre','year',
#                                       'director','writer','star','country','company']).drop('score',axis=1)
#X = pd.get_dummies(movies_df, columns=['rating','genre','year','country']).drop('score',axis=1)
X = movies_df['adjusted_gross']

# Create our target
y = movies_df['score']

In [21]:
#X = X.drop(columns=['director','writer','star','company'], axis=1)
X

0       1.545383e+08
1       1.935170e+08
2       1.770250e+09
3       2.744066e+08
4       1.310202e+08
            ...     
7407    1.163251e+08
7408    4.830985e+08
7409    3.832988e+03
7410    4.327979e+05
7411    1.388922e+04
Name: adjusted_gross, Length: 7408, dtype: float64

In [13]:
numeric_df

,name,gross
0,The Shining,46998772.0
1,The Blue Lagoon,58853106.0
2,Star Wars: Episode V - The Empire Strikes Back,538375067.0
3,Airplane!,83453539.0
4,Caddyshack,39846344.0
...,...,...
7407,The Call of the Wild,111105497.0
7408,The Eight Hundred,461421559.0
7409,The Quarry,3661.0
7410,Tulsa,413378.0


In [14]:
# Merge categorical and numeric columns to create features df (X)
X.merge(numeric_df, on='name')
X

,name,votes,gross,runtime,rating_Approved,rating_G,rating_NC-17,rating_Not Rated,rating_PG,rating_PG-13,...,country_Switzerland,country_Taiwan,country_Thailand,country_Turkey,country_United Arab Emirates,country_United Kingdom,country_United States,country_Vietnam,country_West Germany,country_Yugoslavia
0,The Shining,927000.0,46998772.0,146.0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,The Blue Lagoon,65000.0,58853106.0,104.0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,Star Wars: Episode V - The Empire Strikes Back,1200000.0,538375067.0,124.0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
3,Airplane!,221000.0,83453539.0,88.0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
4,Caddyshack,108000.0,39846344.0,98.0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7407,The Call of the Wild,42000.0,111105497.0,100.0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7408,The Eight Hundred,3700.0,461421559.0,149.0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
7409,The Quarry,2400.0,3661.0,98.0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
7410,Tulsa,294.0,413378.0,120.0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0


In [15]:
# Drop 'name' column
X = X.drop(columns=['name'])
X

,votes,gross,runtime,rating_Approved,rating_G,rating_NC-17,rating_Not Rated,rating_PG,rating_PG-13,rating_R,...,country_Switzerland,country_Taiwan,country_Thailand,country_Turkey,country_United Arab Emirates,country_United Kingdom,country_United States,country_Vietnam,country_West Germany,country_Yugoslavia
0,927000.0,46998772.0,146.0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
1,65000.0,58853106.0,104.0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
2,1200000.0,538375067.0,124.0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
3,221000.0,83453539.0,88.0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
4,108000.0,39846344.0,98.0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7407,42000.0,111105497.0,100.0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
7408,3700.0,461421559.0,149.0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7409,2400.0,3661.0,98.0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
7410,294.0,413378.0,120.0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0


In [16]:
# Check balance of target values
y.value_counts()

6.6    348
6.4    344
6.2    339
6.5    326
6.7    318
      ... 
8.9      3
1.9      3
2.1      2
2.4      2
2.8      2
Name: score, Length: 68, dtype: int64

In [27]:
# Split into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

# Reshape the arrays
X_train = X_train.values.reshape(-1,1)
X_test = X_test.values.reshape(-1,1)
y_train = y_train.values.reshape((-1,1))
y_test = y_test.values.reshape((-1,1))

In [28]:
X.shape

(7408,)

In [29]:
y.shape

(7408,)

In [30]:
# Train the data into Linear Regression algorithm
# Create an instance
lr = LinearRegression()
# Fit into training data
lr.fit(X_train,y_train)

LinearRegression()

In [31]:
# Retrieve the intercept
print(lr.intercept_)

[6.29704348]


In [32]:
# Retrieve the coefficient(s)
print(lr.coef_)

[[9.15466922e-10]]


In [33]:
# Make predictions
y_pred = lr.predict(X_test)

In [34]:
# Determine RMSE (root mean square error)
mse = mean_squared_error(y_test, y_pred, squared=False)
rmse = sqrt(mse)
rmse

0.968789306132263

In [35]:
# Determine the MAE (mean absolute error)
mae = mean_absolute_error(y_test,y_pred)
mae

0.73654718528743

In [36]:
# Determine the r2_score
r2_score = r2_score(y_test,y_pred)
r2_score

0.041693884090032385